In [1]:
old_territory = spark.sql("SELECT ter_concat_territory, market, order_region,order_country, order_state, order_city FROM DQT_Gold_LakeHouse_Layer.dim_territory")


StatementMeta(, 004832a1-e1bc-40ea-a2b3-31ec4ba7e499, 3, Finished, Available)

In [2]:
old_store = spark.sql("SELECT st_concat_store_address, store_country, store_city, store_street, latitude,longitude FROM DQT_Gold_LakeHouse_Layer.dim_store")



StatementMeta(, 004832a1-e1bc-40ea-a2b3-31ec4ba7e499, 4, Finished, Available)

In [3]:
path= "abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/"
territory = spark.read.option("header", "true") .csv(path + "territory")
store = spark.read.option("header", "true") .csv(path + "store")


territory = territory.exceptAll(old_territory)
store =  store.exceptAll(old_store)

StatementMeta(, 004832a1-e1bc-40ea-a2b3-31ec4ba7e499, 5, Finished, Available)

In [4]:
# get max key
territory_df = spark.sql("SELECT MAX(territory_key) as max_territory_key FROM DQT_Gold_LakeHouse_Layer.dim_territory")
store_df = spark.sql("SELECT MAX(store_key) as max_store_key FROM DQT_Gold_LakeHouse_Layer.dim_store")



max_key_territory = territory_df.collect()[0]['max_territory_key']
max_key_store = store_df.collect()[0]['max_store_key']


StatementMeta(, 004832a1-e1bc-40ea-a2b3-31ec4ba7e499, 6, Finished, Available)

In [5]:
from pyspark.sql.functions import current_timestamp, lit, row_number,col
from pyspark.sql.window import Window
from functools import reduce
from pyspark.sql import functions as F

StatementMeta(, 004832a1-e1bc-40ea-a2b3-31ec4ba7e499, 7, Finished, Available)

# create dim_territory

In [9]:

# Define a window specification
window_spec_key = Window.orderBy("ter_concat_territory")  
# Add a new column 'transaction' with consecutive IDs starting from 1
dim_territory = territory.withColumn("territory_key",max_key_territory +  row_number().over(window_spec_key))

dim_territory = dim_territory.select("territory_key", "ter_concat_territory", "market", "order_region",'order_country', "order_state", "order_city")

dim_territory.show()



StatementMeta(, 004832a1-e1bc-40ea-a2b3-31ec4ba7e499, 11, Finished, Available)

+-------------+--------------------+------+------------+-----------+-------------+----------+
|territory_key|ter_concat_territory|market|order_region|order_state|order_country|order_city|
+-------------+--------------------+------+------------+-----------+-------------+----------+
+-------------+--------------------+------+------------+-----------+-------------+----------+



# create store region

In [7]:

# Define a window specification
window_spec_key = Window.orderBy("st_concat_store_address")  
# Add a new column 'transaction' with consecutive IDs starting from 1
dim_store = store.withColumn("store_key", max_key_store +  row_number().over(window_spec_key))

dim_store = dim_store.select("store_key", "st_concat_store_address","store_country", "store_city", "store_street")

display(dim_store)



StatementMeta(, 004832a1-e1bc-40ea-a2b3-31ec4ba7e499, 9, Finished, Available)

SyntaxError: invalid syntax (516569590.py, line 8)

# Load data to gold

## dim_territory

In [ ]:
dim_territory.write.mode("append").format("delta").saveAsTable("dim_territory")

StatementMeta(, , , Cancelled, )

## dim_store_region

In [ ]:
dim_store.write.mode("append").format("delta").saveAsTable("dim_store")

StatementMeta(, , , Cancelled, )